In [13]:
import fitz

input_pdf_path = './base_CV.pdf'
output_pdf_path = './test.pdf'

pdf_document = fitz.open(input_pdf_path)

def list_fields(doc):
    for page_num in range(len(doc)):
        print(f"Page {page_num + 1}")
        page = doc.load_page(page_num)
        for widget in page.widgets():
            print(f"Field name: {widget.field_name}, Field type: {widget.field_type}")

print('Fields before filling:')
list_fields(pdf_document)


Fields before filling:
Page 1
Field name: name, Field type: 7
Field name: disability, Field type: 7
Field name: Overview, Field type: 7
Field name: work, Field type: 7
Field name: education, Field type: 7
Field name: telephone, Field type: 7
Field name: email, Field type: 7
Field name: social, Field type: 7
Field name: address, Field type: 7
Field name: skills, Field type: 7
Field name: award_1, Field type: 7
Field name: award_2, Field type: 7
Field name: award_3, Field type: 7


In [15]:
def fill_fields(doc, data):
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        for widget in page.widgets():
            field_name = widget.field_name
            if field_name in data:
                value = data[field_name]
                if isinstance(value, list):
                    # Check if the list contains dictionaries (for 'work' or similar properties)
                    if all(isinstance(item, dict) for item in value):
                        value_str = '; '.join([
                            f"{item.get('title', 'N/A')} at {item.get('company', 'N/A')} from {item.get('start_date', 'N/A')} to {item.get('end_date', 'N/A')}: {item.get('description', 'N/A')}"
                            for item in value
                        ])
                        widget.update(value=value_str)
                    else:
                        widget.update(value=', '.join(value))
                else:
                    widget.update(value=value)
    return doc

def add_image(doc, image_path, page_num=0, rect=(100, 100, 200, 200)):
    page = doc.load_page(page_num)
    img_rect = fitz.Rect(rect)
    page.insert_image(img_rect, filename=image_path)

# Uncomment the following line if you want to add an image
# add_image(pdf_document, data['image'])

data = {
    'name': 'John Doe',
    'email': 'example@mail.com',
    'telephone': '1234567890',
    'address': '1234 Main Street, City, Country',
    'Overview': 'I am John Doe, an autistic individual with a strong passion for computers. I have developed several skills in programming, troubleshooting, and software development.',
    'skills': ['Python', 'Java', 'C++', 'JavaScript', 'HTML', 'CSS'],
    'disability': 'Autism',
    'social': "https://www.linkedin.com/in/johndoe",
    'work': [
        {
            'title': 'Software Developer',
            'company': 'ABC Inc.',
            'start_date': 'January 2020',
            'end_date': 'Present',
            'description': 'Developed software applications using Python, Java, and C++.'
        },
        {
            'title': 'Technical Support Specialist',
            'company': 'XYZ Corp.',
            'start_date': 'June 2018',
            'end_date': 'December 2019',
            'description': 'Provided technical support to customers and resolved software issues.'
        }
    ],
    'education': [
        {
            'degree': 'Bachelor of Science in Computer Science',
            'school': 'University of ABC',
            'graduation_date': 'May 2018',
            'start_date': 'September 2014',
        }
    ],
}

pdf_document = fill_fields(pdf_document, data)

pdf_document.save(output_pdf_path)
pdf_document.close()

AttributeError: 'Widget' object has no attribute 'set_text'

In [1]:
from fillpdf import fillpdfs


data = {
    'name': 'John Doe',
    'email': 'example@mail.com',
    'telephone': '1234567890',
    'address': 'some address',
    'Overview': 'I am John Doe, an autistic individual with a strong passion for computers. I have developed several skills in programming, troubleshooting, and software development.',
    'skills': ['Python', 'Java', 'C++', 'JavaScript', 'HTML', 'CSS'],
    'disability': 'Autism',
    'social': "some website",
    'work': [
        {
            'title': 'Software Developer',
            'company': 'ABC Inc.',
            'start_date': 'January 2020',
            'end_date': 'Present',
            'description': 'Developed software applications using Python, Java, and C++.'
        },
        {
            'title': 'Technical Support Specialist',
            'company': 'XYZ Corp.',
            'start_date': 'June 2018',
            'end_date': 'December 2019',
            'description': 'Provided technical support to customers and resolved software issues.'
        }
    ],
    'education': [
        {
            'degree': 'Bachelor of Science in Computer Science',
            'school': 'University of ABC',
            'graduation_date': 'May 2018',
            'start_date': 'September 2014',
        }
    ],
    'award_1': 'Hardworking',
    'award_2': 'Dedicated',
    'award_3': 'Understanding',
}

input_pdf_path = './base_CV.pdf'
output_pdf_path = './test2.pdf'

form_fields = list(fillpdfs.get_form_fields(input_pdf_path).keys())
print(form_fields)

def preprocess_data(data):
    flattened_data = {}
    
    for key, value in data.items():
        if isinstance(value, list):
            if key == 'work':
                value_str = '\n\n'.join([
                    f"{item.get('title', 'N/A')} at {item.get('company', 'N/A')} from {item.get('start_date', 'N/A')} to {item.get('end_date', 'N/A')}: {item.get('description', 'N/A')}"
                    for item in value
                ])
                flattened_data[key] = value_str
            elif key == 'education':
                value_str = '\n\n'.join([
                    f"{item.get('degree', 'N/A')} from {item.get('school', 'N/A')} (Start Date: {item.get('start_date', 'N/A')}, Graduation Date: {item.get('graduation_date', 'N/A')})"
                    for item in value
                ])
                flattened_data[key] = value_str
            else:
                flattened_data[key] = '\n'.join(value)
        else:
            flattened_data[key] = value
            
    return flattened_data

data = preprocess_data(data)

fillpdfs.write_fillable_pdf(input_pdf_path=input_pdf_path, output_pdf_path=output_pdf_path, data_dict=data, flatten=True)

['name', 'disability', 'Overview', 'work', 'education', 'telephone', 'email', 'social', 'address', 'skills', 'award_1', 'award_2', 'award_3']
